# measuring accuracy or suitablility of a model
- fitting the same model across different sampling sets to get a better understanding of the accuracy of the model and helps tuning parameters

## 1. cross validation
- holding out a subset of training data to act as test data

### validation set approach
- by randomly dividing a dataset into training or testing data.
- then computing the mean sqaured error of the models
- repeating it 10 times show that in the case of Auto dataset, 2nd degree polynomials seem to work best
- but the range is much larger than the mean squared error for a single split
- tends to overestimate test error
- cross validation addresses these issues.

### leave one out cross-validation
- leave one sample out and train the rest
- find the mean squared error
- low bias, high variance, as it contains most samples in the dataset
- however, with the large overlapping samples outputs tend to be largely correlated.

### k-fold cross validation
- divide into k equal sections
- pick one as testing set and the rest as training set.
- results in k different iterations of the data model
- average mean squared error in each model is measured.
- commonly uses 5 <= k <= 10
- disadvantage: doesn't measure the model at 100% of the dataset.
- if model picks well-performing attributes before fitting the model, a common mistake is to apply k-fold validation after the selection
- instead should apply to both processes of filtering and fitting.
- because the process of picking predictors often picks those that optimise overall MSE/k-fold validation, therefore may only be suited to the specific training set instead of the entire population

### cross validation on classification
- computed as the average error rate of each iteration of training


## 2. Bootstrap

- measures uncertainty associated to a given estimator or machine learning method

e.g. estimate SE of coefficients from a LR fit
- easily applied to range of methods including ones with difficult to obtain variability

### methodology
1. take a dataset Z of n observations
2. create a new sample of size n with sampling by replacement, creating Z*1
3. calculate $\alpha *1$ as the value that minimises the Variance of Z*1
4. repeat many times and compute standard error with all estimates of a*r


# 3. Coding Cross Validation and Bootstrap

In [2]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [3]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

## 3.1 Validation set

In [4]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

In [5]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train.mpg
model = sm.OLS(y_train, X_train)
results = model.fit()

In [6]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid.mpg
valid_pred = results.predict(exog=X_valid)
np.mean((y_valid - valid_pred) ** 2)

20.7554079592286

- estimating error for higher degree polynomials

In [7]:
# create function returning MSE on the test set

def eval_mse(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(exog=X_test)

    return np.mean((y_test - test_pred) ** 2)

In [8]:
# using the function eval_mse to find validation error for degrees of polynomials

mse = np.zeros(3)
for idx, deg in enumerate(range(1, 4)):
    mse[idx] = eval_mse([poly('horsepower', deg)], 'mpg', Auto_train, Auto_valid)
mse

array([20.75540796, 16.94510676, 16.97437833])

## 3.2 Cross-validation

- even though is possible for all packages, sklearn has better support for cross-validation

In [9]:
# sklearn_sm() is a wrapper to enable use of sm models in sklearn

hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))

X, Y = Auto.drop(columns=['mpg']), Auto.mpg
cv_res = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_res['test_score'])
cv_err

24.23151351792923

In [10]:
# fitting for different degree polynomials

cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443034, 19.0332226 ])

In [11]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0)

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])



## 3.3 Bootstrap

In [12]:
portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar=False)
    return ((cov_[1, 1] - cov_[0, 1]) / (cov_[0, 0] + cov_[1, 1] - 2 * cov_[0, 1]))

In [13]:
alpha_func(portfolio, range(100))

0.57583207459283

In [14]:
rng = np.random.default_rng(0)
alpha_func(portfolio, rng.choice(100, 100, replace=True))

0.6074452469619002

In [16]:
# finding standard error
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value ** 2
    return np.sqrt(second_ / B - (first_ / B) ** 2)

In [18]:
alpha_SE = boot_SE(alpha_func, portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277516

- estimating accuracy of LR model
    - write a function taking D and idx as only arguments
    - then return the mean squared error of the model
    - bootstrap specific regression model.

In [22]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [23]:
# using partial() to freeze first two arguments

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [24]:
rng = np.random.default_rng(0)
np.array([hp_func(D=Auto, idx=rng.choice(392, 392, replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [25]:
# use boot SE to find the SE for the 10 bootstrap sets

hp_se = boot_SE(hp_func, Auto, B=1000, seed=0)
hp_se

intercept     0.857854
horsepower    0.007458
dtype: float64